In [ ]:
!pip install --q lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 10.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from lifelines import CoxPHFitter

data = [
    # property_id, status, start_date, end_date
    ("P001", "leased",      "2018-01-01", "2020-02-28"),
    ("P001", "maintenance", "2020-03-01", "2020-05-01"),
    ("P001", "listed",      "2020-05-02", "2020-06-15"),  # ends with lease next
    ("P001", "leased",      "2020-06-16", "2022-05-31"),
    ("P001", "listed",      "2022-06-01", "2022-08-01"),  # censored (no new lease yet)
    ("P001", "leased",      "2022-08-02", "2024-09-01"),
    ("P001", "maintenance", "2024-09-02", "2024-11-01"),
    ("P001", "listed",      "2024-11-02", "2024-12-31"),

    ("P002", "leased",      "2019-02-01", "2021-03-31"),
    ("P002", "listed",      "2021-04-01", "2021-06-30"),  # ends with lease next
    ("P002", "leased",      "2021-07-01", "2023-01-31"),
    ("P002", "listed",      "2023-02-01", "2023-07-01"),  # ends with maintenance → censored
    ("P002", "maintenance", "2023-07-02", "2023-08-01"),
    ("P002", "listed",      "2023-08-02", "2023-09-15"),
    ("P002", "leased",      "2023-09-16", "2024-12-31"),

    ("P003", "leased",      "2020-05-01", "2021-12-31"),
    ("P003", "listed",      "2022-01-01", "2022-03-15"),  # ends with lease next
    ("P003", "leased",      "2022-03-16", "2024-09-30"),
    ("P003", "maintenance", "2024-10-01", "2024-11-30"),
    ("P003", "listed",      "2024-12-01", "2024-12-31"),
]

statuses = pd.DataFrame(data, columns=["property_id","status","start_date","end_date"])
statuses["start_date"] = pd.to_datetime(statuses["start_date"])
statuses["end_date"]   = pd.to_datetime(statuses["end_date"])
statuses

,property_id,status,start_date,end_date
0,P001,leased,2018-01-01,2020-02-28
1,P001,maintenance,2020-03-01,2020-05-01
2,P001,listed,2020-05-02,2020-06-15
3,P001,leased,2020-06-16,2022-05-31
4,P001,listed,2022-06-01,2022-08-01
5,P001,leased,2022-08-02,2024-09-01
6,P001,maintenance,2024-09-02,2024-11-01
7,P001,listed,2024-11-02,2024-12-31
8,P002,leased,2019-02-01,2021-03-31
9,P002,listed,2021-04-01,2021-06-30


In [ ]:
risk_statuses = {"listed"}
records = []

for pid, g in statuses.groupby("property_id", sort=False):
  g = g.sort_values('start_date').reset_index(drop=True)
  for i, row in g.iterrows():
    if row['status'] in risk_statuses:
      spell_start = row['start_date']
      if i + 1 < len(g):
        next_row = g.iloc[i + 1]
        spell_end = next_row['start_date']
        event = 1 if next_row['status'] == 'leased' else 0
      else:
        spell_end = row['end_date']
        event = 0

      duration = (spell_end - spell_start).days
      records.append({
          "property_id": pid,
          "spell_start": spell_start,
          "spell_end": spell_end,
          "duration": duration,
          "event": event
      })

df = pd.DataFrame(records)
df

,property_id,spell_start,spell_end,duration,event
0,P001,2020-05-02,2020-06-16,45,1
1,P001,2022-06-01,2022-08-02,62,1
2,P001,2024-11-02,2024-12-31,59,0
3,P002,2021-04-01,2021-07-01,91,1
4,P002,2023-02-01,2023-07-02,151,0
5,P002,2023-08-02,2023-09-16,45,1
6,P003,2022-01-01,2022-03-16,74,1
7,P003,2024-12-01,2024-12-31,30,0


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['duration'].plot(kind='hist', bins=20, title='duration')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['event'].plot(kind='hist', bins=20, title='event')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('property_id').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['property_id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='property_id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['property_id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='duration', y='property_id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['property_id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='event', y='property_id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)